**CASE 2**

In [9]:
### Library Imports ###

import pandas as pd
import numpy as np
from scipy.optimize import brentq

In [10]:
### Data Setup ###

LGD = 0.40
r = 0.03
maturities = np.array([1, 3, 5, 7, 10])
cds_rates_bps = np.array([100, 110, 120, 120, 125])
cds_rates = cds_rates_bps / 10000

**Q1**

In [11]:
##### Q1 #####

# Average Hazard Rates
average_hazard_rates_q1 = cds_rates / LGD

# Forward Hazard Rates
forward_hazard_rates_q1 = np.zeros(len(maturities))
forward_hazard_rates_q1[0] = average_hazard_rates_q1[0]
for i in range(1, len(maturities)):
    numerator = average_hazard_rates_q1[i] * maturities[i] - average_hazard_rates_q1[i-1] * maturities[i-1]
    forward_hazard_rates_q1[i] = numerator / (maturities[i] - maturities[i-1])

# Default Probabilities
survival_probs_q1 = np.exp(-average_hazard_rates_q1 * maturities)
default_probs_q1 = np.zeros(len(maturities))
default_probs_q1[0] = 1 - survival_probs_q1[0]
for i in range(1, len(maturities)):
    default_probs_q1[i] = survival_probs_q1[i-1] - survival_probs_q1[i]

results_q1 = pd.DataFrame({
    'Maturity': [f'{m}Y' for m in maturities],
    'CDS Rate (bps)': cds_rates_bps,
    '(Average) Hazard Rate': average_hazard_rates_q1,
    'Forward Hazard Rate': forward_hazard_rates_q1,
    'Default Probability': default_probs_q1
})
print("\n", results_q1.to_string(index=False))


 Maturity  CDS Rate (bps)  (Average) Hazard Rate  Forward Hazard Rate  Default Probability
      1Y             100                0.02500             0.025000             0.024690
      3Y             110                0.02750             0.028750             0.054498
      5Y             120                0.03000             0.033750             0.060103
      7Y             120                0.03000             0.030000             0.050124
     10Y             125                0.03125             0.034167             0.078969


**Q2**

In [12]:
##### Q2 #####

def survival_probability(t, lambda_periods):
    integral = 0.0
    for t_start, t_end, lam in lambda_periods:
        if t <= t_start:
            break
        elif t <= t_end:
            integral += lam * (t - t_start)
            break
        else:
            integral += lam * (t_end - t_start)
    return np.exp(-integral)

def price_cds(maturity, cds_rate, lgd, r, lambda_periods):
    num_quarters = int(maturity * 4)
    payment_times = np.linspace(0.25, maturity, num_quarters)
    
    premium_leg = 0.0
    for i, t_i in enumerate(payment_times):
        t_prev = 0 if i == 0 else payment_times[i-1]
        dt = t_i - t_prev
        q_surv = survival_probability(t_i, lambda_periods)
        df = np.exp(-r * t_i)
        premium_leg += cds_rate * df * dt * q_surv
    
    for i, t_i in enumerate(payment_times):
        t_prev = 0 if i == 0 else payment_times[i-1]
        dt = t_i - t_prev
        t_mid = (t_i + t_prev) / 2
        q_prev = survival_probability(t_prev, lambda_periods)
        q_i = survival_probability(t_i, lambda_periods)
        df_mid = np.exp(-r * t_mid)
        premium_leg += cds_rate * df_mid * (q_prev - q_i) * (dt / 2)
    
    protection_leg = 0.0
    for i, t_i in enumerate(payment_times):
        t_prev = 0 if i == 0 else payment_times[i-1]
        t_mid = (t_i + t_prev) / 2
        q_prev = survival_probability(t_prev, lambda_periods)
        q_i = survival_probability(t_i, lambda_periods)
        df_mid = np.exp(-r * t_mid)
        protection_leg += lgd * df_mid * (q_prev - q_i)
    
    return premium_leg - protection_leg

# Strip forward hazard rates
forward_lambdas_q2 = []
lambda_periods = []

for idx in range(len(maturities)):
    mat = maturities[idx]
    def objective(lam):
        temp_periods = lambda_periods.copy()
        t_start = 0 if idx == 0 else maturities[idx-1]
        temp_periods.append((t_start, mat, lam))
        return price_cds(mat, cds_rates[idx], LGD, r, temp_periods)
    
    solution = brentq(objective, 0.0001, 1.0)
    forward_lambdas_q2.append(solution)
    t_start = 0 if idx == 0 else maturities[idx-1]
    lambda_periods.append((t_start, mat, solution))

# Calculate average hazard rates and default probabilities
average_hazard_rates_q2 = []
default_probs_q2 = []
for idx in range(len(maturities)):
    mat = maturities[idx]
    integral = sum(lam * (min(t_end, mat) - t_start) 
                   for t_start, t_end, lam in lambda_periods if t_start < mat)
    average_hazard_rates_q2.append(integral / mat)
    
    q_surv = survival_probability(mat, lambda_periods)
    q_prev = 1.0 if idx == 0 else survival_probability(maturities[idx-1], lambda_periods)
    default_probs_q2.append(q_prev - q_surv)

results_q2 = pd.DataFrame({
    'Maturity': [f'{m}Y' for m in maturities],
    'CDS Rate (bps)': cds_rates_bps,
    '(Average) Hazard Rate': average_hazard_rates_q2,
    'Forward Hazard Rate': forward_lambdas_q2,
    'Forward Default Probability': default_probs_q2
})
print("\n", results_q2.to_string(index=False))



 Maturity  CDS Rate (bps)  (Average) Hazard Rate  Forward Hazard Rate  Forward Default Probability
      1Y             100               0.024907             0.024907                     0.024599
      3Y             110               0.027472             0.028754                     0.054512
      5Y             120               0.030179             0.034239                     0.060950
      7Y             120               0.030096             0.029888                     0.049898
     10Y             125               0.031604             0.035124                     0.081013


**Q3**

In [13]:
##### Q3 #####

# Calculate 7Y CDS legs
mat_7y, cds_rate_7y = 7, cds_rates[3]
payment_times = np.linspace(0.25, mat_7y, int(mat_7y * 4))

premium_leg = sum(cds_rate_7y * np.exp(-r * t_i) * (t_i - (0 if i == 0 else payment_times[i-1])) * 
                  survival_probability(t_i, lambda_periods) 
                  for i, t_i in enumerate(payment_times))

premium_leg += sum(cds_rate_7y * np.exp(-r * (t_i + (0 if i == 0 else payment_times[i-1]))/2) * 
                   (survival_probability(0 if i == 0 else payment_times[i-1], lambda_periods) - 
                    survival_probability(t_i, lambda_periods)) * 
                   (t_i - (0 if i == 0 else payment_times[i-1])) / 2
                   for i, t_i in enumerate(payment_times))

protection_leg = sum(LGD * np.exp(-r * (t_i + (0 if i == 0 else payment_times[i-1]))/2) * 
                     (survival_probability(0 if i == 0 else payment_times[i-1], lambda_periods) - 
                      survival_probability(t_i, lambda_periods))
                     for i, t_i in enumerate(payment_times))

print(f"\nPremium Leg: {premium_leg:.10f}")
print(f"Protection Leg: {protection_leg:.10f}")
print(f"Difference: {abs(premium_leg - protection_leg):.2e}")
print(f"Result:{' Lower than tolerance' if abs(premium_leg - protection_leg) < 1e-6 else 'Failed'}")


Premium Leg: 0.0685500545
Protection Leg: 0.0685500545
Difference: 2.01e-14
Result: Lower than tolerance
